In [1]:
## Import all the packages
import json
import pandas as pd
import numpy as np

from transformers import pipeline
from flask import Flask, request, jsonify
!pip install -U -q transformers accelerate
! pip install diffusers transformers accelerate torch torchvision

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from diffusers import StableDiffusionPipeline
import ipywidgets as widgets
from IPython.display import display, HTML

import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
## Upload the file customer_features_clustering_detai.csv to google colab
from google.colab import files
uploaded = files.upload()

Saving customer_features_clustering_details.csv to customer_features_clustering_details (1).csv


In [3]:
## Read csv file and display 1 rows
profile_cluster = pd.read_csv("customer_features_clustering_details.csv")
print(profile_cluster.head(1))

   client_id  total_spending  avg_transaction  transaction_count  \
0          0        50852.09        47.703649               1066   

   Chip Transaction  Online Transaction  Swipe Transaction  past_3_months_sum  \
0               958                  89                 19           15724.66   

   past_6_months_sum  past_9_months_sum  ...  per_capita_income  \
0           28900.12            43981.0  ...            29237.0   

   yearly_income  total_debt  credit_score  num_credit_cards  \
0        59613.0     36199.0           763                 4   

   debt_to_income_ratio  cluster    tsne_1    tsne_2  \
0              0.607233        5  2.707905  7.505155   

                                             Profile  
0  Clustername: Older, Conservative Spenders; Spe...  

[1 rows x 59 columns]


In [4]:
## Remove columns from dataframe as they won't be useful for email, tagline and profile generation
profile_cluster = profile_cluster.drop(columns=['tsne_1', 'tsne_2','cluster'])

## Fetch a customer's profile based on their client_id from a DataFrame

In [18]:
# Function to fetch customer profile
def get_customer_profile(client_id, profile_cluster):
    """
    Fetches the profile summary of a customer based on their client_id.

    Args:
        client_id (int): The ID of the customer.
        profile_cluster (pd.DataFrame): DataFrame containing customer profiles.

    Returns:
        dict or None: A dictionary with the customer's profile details, or None if not found.
    """
    try:
        # Filter the DataFrame for the given client_id
        customer_data = profile_cluster[profile_cluster["client_id"] == client_id]

        if customer_data.empty:
            print(f"No customer found with client_id: {client_id}")
            return None

        # Convert the row to a dictionary and return
        profile_summary = customer_data.iloc[0].to_dict()
        return profile_summary

    except KeyError:
        print("The DataFrame does not contain a 'client_id' column.")
        return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## Function dynamically creates a summary string (or prompt) for a user's profile based on their data

In [43]:
def generate_profile_content(profile):
    """
    Generates a marketing content summary based on the provided customer profile.

    Args:
        profile (dict): A dictionary containing customer profile details.

    Returns:
        str: A summary string describing the customer's profile.
    """
    if not profile:
        return "Invalid profile data provided."

    # Extract variables with default values
    user_profile = profile.get("Profile", "N/A")
    spending_pattern = profile.get("total_spending", "N/A")
    transaction_types = profile.get("top_5_merchant_categories", "N/A")
    count_transactions = profile.get("transaction_count", "N/A")
    past_3_months_purchase = profile.get("past_3_months_sum", "N/A")
    customer_retirement_age = profile.get("retirement_age", "N/A")
    credit_score = profile.get("credit_score", "N/A")
    yearly_income = profile.get("yearly_income", "N/A")
    gender_customer = profile.get("gender", "N/A")
    age_group = profile.get("current_age", "N/A")
    num_credit_cards = profile.get("num_credit_cards", "N/A")
    growth_rate = profile.get("growth_rate_3m_vs_6m", "N/A")

    # Create the profile summary
    profile_summary = (
        f"User Profile: {user_profile}; Gender: {gender_customer}; Age: {age_group}; "
        f"Spending Pattern: {spending_pattern}; Transaction Types: {transaction_types}; "
        f"Total Transactions: {count_transactions}; Past 3 Months Transactions: {past_3_months_purchase}; "
        f"Retirement Age: {customer_retirement_age}; Credit Score: {credit_score}; "
        f"Yearly Income: {yearly_income}; Credit Cards: {num_credit_cards}; "
        f"Growth Transactions (3M vs 6M): {growth_rate}."
    )

    return profile_summary

##  We will use Phi-3 model for text generation , load the model from Huggingface


In [7]:
# --- LLM Model Setup ---
# LLM model we'll be using
model_name = "microsoft/Phi-3-mini-4k-instruct"


# Configure the model for inference
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Automatically use available GPU
    torch_dtype=torch.float16,  # Can improve performance on some GPUs (using FP16 precision)
    trust_remote_code=True,  # Trust code from remote repository
)

# Load the tokenizer for the chosen model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a pipeline object for text generation with the LLM
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [8]:
# Parameters to control LLM's response generation
generation_args = {
    "max_new_tokens": 600,     # Maximum length of the response
    "return_full_text": False, # Only return the generated text
}

In [9]:
## Use this to run over the LLM model by passing over the prompt to generate the output text
def query(messages):
  """
  Sends a conversation history to the LLM model and returns the answer

  Args:
      messages (list): A list of dictionaries, each with "role" and "content" keys

  Returns:
      str: The answer from the AI assistant.
  """

  # Send the full conversation history to the LLM
  output = pipe(messages, **generation_args)

  # Extract the response
  return output[0]['generated_text']

## Function to generate the prompt for email content generation. We need to pass summary string describing the customer's profile. Funtion to return generate email reponse for a given prompt

In [10]:
def Marketing_email_genereator(customer_profile_dict):
    messages = [
        {"role": "system", "content": """
            You are a professional email copywriter specializing in personalized marketing content for the banking sector.
            Your task is to generate concise, engaging, and customer-specific email content that highlights the benefits of banking products
            while resonating with the recipient's profile.

            The email should:
            1. Address the recipient personally, using their name or appropriate salutation.
            2. Reflect their specific financial needs, goals, or preferences based on their profile information.
            3. Be professional yet approachable, avoiding jargon and ensuring clarity.
            4. Stay within 150–200 words for optimal readability.
            5. Only add details for banking-related products.
            6. Maintain a tone that aligns with the bank's branding, focusing on trust, security, and customer empowerment

            Focus on making the email informative, yet aspirational, motivating the recipient to engage with the bank's offerings. The email should reflect the customer's profile without going into unnecessary details about the profile.
        """},
        {"role": "user", "content": f"Use this customer profile to generate a personalized marketing email: {customer_profile_dict}."}
    ]

    result = query(messages)
    return result

## Function to generate the prompt to come up with Marketing Tagline and the Model response

In [52]:
def generate_marketing_tagline(customer_profile_dict):
    messages = [
        {"role": "system", "content": """
            You are an expert creative marketing strategist specializing in crafting impactful and personalized taglines for banking customers.
            Your task is to generate concise, engaging, and customer-specific taglines that resonate with the recipient's unique financial profile.

            The tagline should:
            1. Be between 8–12 words long for maximum impact and readability.
            2. Use a positive, professional, and aspirational tone to inspire confidence in banking services.
            3. Incorporate personalized elements such as age group, spending habits, financial growth, or unique strengths derived from the customer's profile.
            4. Be actionable, suggesting a positive financial outcome or benefit (e.g., “Maximize Your Savings Potential,” “Secure Your Retirement Goals with Confidence”).
            5. Align with the bank’s professional yet approachable branding.

            Ensure that taglines are distinctly tailored to the customer's financial goals or behaviors, making them feel directly addressed and understood.
        """},
        {"role": "user", "content": f"""
            Based on the following customer details, create 2–3 highly personalized and varied taglines that reflect their financial profile:
            - Age: {customer_profile_dict.get('current_age', 'N/A')}
            - Gender: {customer_profile_dict.get('gender', 'N/A')}
            - Annual Spending: {customer_profile_dict.get('total_spending', 'N/A')}
            - Credit Score: {customer_profile_dict.get('credit_score', 'N/A')}
            - Spending Patterns: {customer_profile_dict.get('top_5_merchant_categories', 'N/A')}
            - Number Credit Cards: {customer_profile_dict.get('num_credit_cards', 'N/A')}
        """}
    ]

    result = query(messages)
    return result

## Function to generate customer profile prompt and LLM model response

In [20]:
def Customer_profile_summary(customer_profile_dict):
    messages = [
        {"role": "system", "content": """
            You are a professional financial analyst specializing in creating clear and concise customer profile summaries for banking clients.
            Your task is to generate a summary of no more than 100 words, based on the details provided.

            The summary should:
            1. Use the customer’s personal details, financial data, and cluster profile insights to create a holistic and engaging profile.
            2. Ensure the tone is professional, concise, and specific, focusing on actionable insights or recommendations.
            3. Avoid unnecessary jargon and keep the language simple and approachable.
            4. Highlight key attributes from the cluster profile data that would be provided.

            Make sure you suggest the summary, you don't need to show over the data details present in the shared data from the user, only the customer summary.
        """},
        {"role": "user", "content": f"Use this customer details to come up with customer profile summary: {customer_profile_dict}."}
    ]

    result = query(messages)
    return result

#user_profile_summary = Customer_profile_summary(example1)

## We will use Stable diffusion to create Images based on the text provided

In [14]:
# Load the Stable Diffusion model
pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipeline.to("cuda")  # Use GPU for faster generation

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.31.0",
  "_name_or_path": "runwayml/stable-diffusion-v1-5",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

## We will condense the customer summary generated by the LLM model into 1-2 concise sentences and use this as a prompt for the Stable Diffusion model to generate the marketing image

In [25]:
def profile_summary_Image(customer_profile):
    messages = [
        {"role": "system", "content": """
         You are a highly advanced image generation model trained to create visual representations based on customer profiles. Your task is to take detailed customer descriptions and
          generate a corresponding image for marketing based on the characteristics mentioned in the text

        """},
        {"role": "user", "content": f"Use this customer profile to come up with 1 sentences max for Image generation: {customer_profile}."}
    ]

    result = query(messages)
    return result

In [31]:
# Pass in profile summary to comeup with the image
def generate_image_prompt(profile_summary):
    return (
        f"""Create a professional marketing poster for a retail bank, highlighting financial insights and promotional offers. Use modern, clean infographics, yellow accents, and icons
        for spending categories, transaction growth, and engagement metrics
        Customer Profile Summary: {profile_summary}"""
    )

# Creating a Simple UI, where we can pass the client_id based on which the LLM model will generate customer profile summary , marketing email , Tagline and Image

In [51]:
# Display the title for the UI with HTML formatting to highlight it
display(HTML("<h1 style='color: #4CAF50;'>AI-Driven Personalized Marketing Content Generation</h1>"))

# Define the input field for Client ID
client_id_input = widgets.BoundedIntText(
    description="Client ID:",  # Label for the input field
    placeholder="Enter client ID (e.g., client1)",  # Placeholder text
    max=10000,  # Set a high upper limit (or adjust as needed)
)

# Define the button to trigger response generation
generate_button = widgets.Button(description="Generate Responses")

# Output area to display results
output_area = widgets.Output()

# Define the event handler for when the button is clicked
def on_button_click(b):
    with output_area:
        output_area.clear_output()  # Clear previous outputs

        # Retrieve the value entered for client_id
        client_id = client_id_input.value

        # Check if the entered client_id is valid
        if int(client_id) in list(profile_cluster['client_id'].unique()):
            profile_summary = get_customer_profile(client_id, profile_cluster)  # Fetch customer profile details
            print("Valid Client ID, generating responses for:", profile_summary['client_id'])

            # Generate customer profile summary
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized customer summary for the user----------")
            example_user = generate_profile_content(profile_summary)
            user_profile_summary = Customer_profile_summary(example_user)
            print("\n--- Customer Summary ---\n", user_profile_summary)

            # Generate personalized marketing email
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized Marketing email for the user-----------")
            email = Marketing_email_genereator(example_user)
            print("\n--- Marketing Email ---\n", email)

            # Generate personalized marketing tagline
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized Marketing tagline for the user---------")
            tagline = generate_marketing_tagline(profile_summary)
            print("\n--- Marketing Tagline ---\n", tagline)

            # Generate customer summary for Image model and run Stable diffusion model for Image generation
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------------Generate Marketing Image for the user--------------------")
            summary_as_string = json.dumps(user_profile_summary)
            summary = profile_summary_Image(summary_as_string)
            image_prompt = generate_image_prompt(summary)
            # Generate the image
            image = pipeline(image_prompt).images[0]
            display(image)


        else:
            # Handle invalid client_id
            print("Invalid Client ID. Please try again.")

# Link the button to the event handler
generate_button.on_click(on_button_click)

# Display the input field, button, and output area in the notebook
display(client_id_input, generate_button, output_area)


BoundedIntText(value=0, description='Client ID:', max=10000)

Button(description='Generate Responses', style=ButtonStyle())

Output()

## Showchasing multiple examples from LLM model

In [54]:
# Display the title for the UI with HTML formatting to highlight it
display(HTML("<h1 style='color: #4CAF50;'>AI-Driven Personalized Marketing Content Generation</h1>"))

# Define the input field for Client ID
client_id_input = widgets.BoundedIntText(
    description="Client ID:",  # Label for the input field
    placeholder="Enter client ID (e.g., client1)",  # Placeholder text
    max=10000,  # Set a high upper limit (or adjust as needed)
)

# Define the button to trigger response generation
generate_button = widgets.Button(description="Generate Responses")

# Output area to display results
output_area = widgets.Output()

# Define the event handler for when the button is clicked
def on_button_click(b):
    with output_area:
        output_area.clear_output()  # Clear previous outputs

        # Retrieve the value entered for client_id
        client_id = client_id_input.value

        # Check if the entered client_id is valid
        if int(client_id) in list(profile_cluster['client_id'].unique()):
            profile_summary = get_customer_profile(client_id, profile_cluster)  # Fetch customer profile details
            print("Valid Client ID, generating responses for:", profile_summary['client_id'])

            # Generate customer profile summary
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized customer summary for the user----------")
            example_user = generate_profile_content(profile_summary)
            user_profile_summary = Customer_profile_summary(example_user)
            print("\n--- Customer Summary ---\n", user_profile_summary)

            # Generate personalized marketing email
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized Marketing email for the user-----------")
            email = Marketing_email_genereator(example_user)
            print("\n--- Marketing Email ---\n", email)

            # Generate personalized marketing tagline
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized Marketing tagline for the user---------")
            tagline = generate_marketing_tagline(profile_summary)
            print("\n--- Marketing Tagline ---\n", tagline)

            # Generate customer summary for Image model and run Stable diffusion model for Image generation
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------------Generate Marketing Image for the user--------------------")
            summary_as_string = json.dumps(user_profile_summary)
            summary = profile_summary_Image(summary_as_string)
            image_prompt = generate_image_prompt(summary)
            # Generate the image
            image = pipeline(image_prompt).images[0]
            display(image)


        else:
            # Handle invalid client_id
            print("Invalid Client ID. Please try again.")

# Link the button to the event handler
generate_button.on_click(on_button_click)

# Display the input field, button, and output area in the notebook
display(client_id_input, generate_button, output_area)


BoundedIntText(value=0, description='Client ID:', max=10000)

Button(description='Generate Responses', style=ButtonStyle())

Output()

In [55]:
# Display the title for the UI with HTML formatting to highlight it
display(HTML("<h1 style='color: #4CAF50;'>AI-Driven Personalized Marketing Content Generation</h1>"))

# Define the input field for Client ID
client_id_input = widgets.BoundedIntText(
    description="Client ID:",  # Label for the input field
    placeholder="Enter client ID (e.g., client1)",  # Placeholder text
    max=10000,  # Set a high upper limit (or adjust as needed)
)

# Define the button to trigger response generation
generate_button = widgets.Button(description="Generate Responses")

# Output area to display results
output_area = widgets.Output()

# Define the event handler for when the button is clicked
def on_button_click(b):
    with output_area:
        output_area.clear_output()  # Clear previous outputs

        # Retrieve the value entered for client_id
        client_id = client_id_input.value

        # Check if the entered client_id is valid
        if int(client_id) in list(profile_cluster['client_id'].unique()):
            profile_summary = get_customer_profile(client_id, profile_cluster)  # Fetch customer profile details
            print("Valid Client ID, generating responses for:", profile_summary['client_id'])

            # Generate customer profile summary
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized customer summary for the user----------")
            example_user = generate_profile_content(profile_summary)
            user_profile_summary = Customer_profile_summary(example_user)
            print("\n--- Customer Summary ---\n", user_profile_summary)

            # Generate personalized marketing email
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized Marketing email for the user-----------")
            email = Marketing_email_genereator(example_user)
            print("\n--- Marketing Email ---\n", email)

            # Generate personalized marketing tagline
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized Marketing tagline for the user---------")
            tagline = generate_marketing_tagline(profile_summary)
            print("\n--- Marketing Tagline ---\n", tagline)

            # Generate customer summary for Image model and run Stable diffusion model for Image generation
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------------Generate Marketing Image for the user--------------------")
            summary_as_string = json.dumps(user_profile_summary)
            summary = profile_summary_Image(summary_as_string)
            image_prompt = generate_image_prompt(summary)
            # Generate the image
            image = pipeline(image_prompt).images[0]
            display(image)


        else:
            # Handle invalid client_id
            print("Invalid Client ID. Please try again.")

# Link the button to the event handler
generate_button.on_click(on_button_click)

# Display the input field, button, and output area in the notebook
display(client_id_input, generate_button, output_area)


BoundedIntText(value=0, description='Client ID:', max=10000)

Button(description='Generate Responses', style=ButtonStyle())

Output()

In [56]:
# Display the title for the UI with HTML formatting to highlight it
display(HTML("<h1 style='color: #4CAF50;'>AI-Driven Personalized Marketing Content Generation</h1>"))

# Define the input field for Client ID
client_id_input = widgets.BoundedIntText(
    description="Client ID:",  # Label for the input field
    placeholder="Enter client ID (e.g., client1)",  # Placeholder text
    max=10000,  # Set a high upper limit (or adjust as needed)
)

# Define the button to trigger response generation
generate_button = widgets.Button(description="Generate Responses")

# Output area to display results
output_area = widgets.Output()

# Define the event handler for when the button is clicked
def on_button_click(b):
    with output_area:
        output_area.clear_output()  # Clear previous outputs

        # Retrieve the value entered for client_id
        client_id = client_id_input.value

        # Check if the entered client_id is valid
        if int(client_id) in list(profile_cluster['client_id'].unique()):
            profile_summary = get_customer_profile(client_id, profile_cluster)  # Fetch customer profile details
            print("Valid Client ID, generating responses for:", profile_summary['client_id'])

            # Generate customer profile summary
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized customer summary for the user----------")
            example_user = generate_profile_content(profile_summary)
            user_profile_summary = Customer_profile_summary(example_user)
            print("\n--- Customer Summary ---\n", user_profile_summary)

            # Generate personalized marketing email
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized Marketing email for the user-----------")
            email = Marketing_email_genereator(example_user)
            print("\n--- Marketing Email ---\n", email)

            # Generate personalized marketing tagline
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------Generating personalized Marketing tagline for the user---------")
            tagline = generate_marketing_tagline(profile_summary)
            print("\n--- Marketing Tagline ---\n", tagline)

            # Generate customer summary for Image model and run Stable diffusion model for Image generation
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("----------------------------------------------------------------------")
            print ("-------------Generate Marketing Image for the user--------------------")
            summary_as_string = json.dumps(user_profile_summary)
            summary = profile_summary_Image(summary_as_string)
            image_prompt = generate_image_prompt(summary)
            # Generate the image
            image = pipeline(image_prompt).images[0]
            display(image)


        else:
            # Handle invalid client_id
            print("Invalid Client ID. Please try again.")

# Link the button to the event handler
generate_button.on_click(on_button_click)

# Display the input field, button, and output area in the notebook
display(client_id_input, generate_button, output_area)


BoundedIntText(value=0, description='Client ID:', max=10000)

Button(description='Generate Responses', style=ButtonStyle())

Output()